# **DELAY PREDICTION**
In this notebook several models are implemented and tested out for predicting the delay of a sbb connection (= sequence of stops in the sbb network). They all output a numeric value which is the delay in minutes for the inputted connection.import os
import pandas as pd
pd.set_option("display.max_columns", 50)
import matplotlib.pyplot as plt
import warnings
import plotly.express as px
import plotly.graph_objects as go
warnings.simplefilter(action='ignore', category=UserWarning)

In [8]:
import os
import pandas as pd
pd.set_option("display.max_columns", 50)
import matplotlib.pyplot as plt
import warnings
import plotly.express as px
import plotly.graph_objects as go
warnings.simplefilter(action='ignore', category=UserWarning)
from pyhive import hive

In [9]:
# Set python variables from environment variables
username = os.environ['USERNAME']
hive_host = os.environ['HIVE_SERVER2'].split(':')[0]
hive_port = os.environ['HIVE_SERVER2'].split(':')[1]

In [10]:
# create connection
conn = hive.connect(
    host=hive_host,
    port=hive_port,
    # auth="KERBEROS",
    # kerberos_service_name = "hive"
)

# create cursor
cur = conn.cursor()

print(f"your username is {username}")
print(f"you are connected to {hive_host}:{hive_port}")

your username is hanygeor
you are connected to iccluster044.iccluster.epfl.ch:10000


In [11]:
query = f"""
    SELECT month AS month_year, LOWER(produkt_id) AS ttype, COUNT(*) AS stops
    FROM {username}.sbb_orc 
    WHERE betriebstag LIKE '__.__.2022' AND produkt_id IS NOT NULL AND produkt_id!=''
    GROUP BY LOWER(produkt_id), month
    ORDER BY LOWER(produkt_id) ASC, month ASC 
"""
data = pd.read_sql(query, conn)

# Cleaning and data preparation
Consider tasks and its assumptions:
   1. We only consider departure and arrival stops in a 15km radius of Zürich's train station, Zürich HB (8503000), (lat, lon) = (47.378177, 8.540192)
   2. We only consider journeys at reasonable hours of the day, and on a typical business day, and assuming a recent schedule. --> TODO think: does this mean we should train our model without weekend days (= typical business day), connections during the night (=reasonable hours of the day) and a recent schedule (which means what??)

In [12]:
# filter for stops in a 15km radius of ZH train station (lat, lon) = (47.378177, 8.540192)
query_filter_stops_near_zh = f"""
    SELECT *
    FROM {username}.sbb_stops_orc
    WHERE (6371 * acos(cos(radians(STOP_LAT)) * cos(radians(47.378177))
                    * cos(radians(8.540192) - radians(STOP_LON))
                    + sin(radians(STOP_LAT)) * sin(radians(47.378177)))) <= 15
"""
stops_near_zh_central = pd.read_sql(query_filter_stops_near_zh, conn)
stops_near_zh_central

,sbb_stops_orc.stop_id,sbb_stops_orc.stop_name,sbb_stops_orc.stop_lat,sbb_stops_orc.stop_lon,sbb_stops_orc.location_type,sbb_stops_orc.parent_station
0,176,Zimmerberg-Basistunnel,47.351677,8.521957,,
1,8500926,"Oetwil a.d.L., Schweizäcker",47.423626,8.403183,,
2,8502075,"Zürich Flughafen, Carterminal",47.451023,8.563729,,
3,8502186,Dietikon Stoffelbach,47.393326,8.398960,,Parent8502186
4,8502186:0,Dietikon Stoffelbach,47.393400,8.398943,,Parent8502186
...,...,...,...,...,...,...
2117,Parent8587020,Bahnhof,47.405834,8.404521,1,
2118,Parent8587651,Glatt (Bus),47.409210,8.595548,1,
2119,Parent8590279,Shopping Center,47.420450,8.368517,1,
2120,Parent8590464,Bahnhof,47.311783,8.524311,1,


## Statistical model
Delay in minutes of a connection A to B is regarded as random variable. In this model we find the average delay with alpha 5% certainty. 

In [13]:
data

,month_year,ttype,stops
0,1,bus,46549800
1,2,bus,46419778
2,3,bus,52291730
3,4,bus,48370016
4,5,bus,49417487
...,...,...,...
70,8,zug,4599227
71,9,zug,4642249
72,10,zug,4913243
73,11,zug,4493776


In [14]:
# set parameters
start = 0
destination = 1

In [15]:
# import data from hdfs using hive (gives an
query = f"""
    SELECT ANKUNFTSZET, AN_PROGNOSE, ABFAHRTSZEIT, AB_PROGNOSE
    FROM {username}.sbb_orc 
"""

data = pd.read_sql(query, conn)

DatabaseError: Execution failed on sql: 
    SELECT ANKUNFTSZET, AN_PROGNOSE, ABFAHRTSZEIT, AB_PROGNOSE
    FROM hanygeor.sbb_orc 

TExecuteStatementResp(status=TStatus(statusCode=3, infoMessages=["*org.apache.hive.service.cli.HiveSQLException:Error while compiling statement: FAILED: SemanticException [Error 10004]: Line 2:11 Invalid table alias or column reference 'ANKUNFTSZET': (possible column names are: betriebstag, fahrt_bezichner, betreiber_id, betreiber_abk, betreiber_name, produkt_id, linien_id, linien_text, umlauf_id, verkehrsmittel_text, zusatzfahrt_tf, faellt_aus_tf, bpuic, haltestellen_name, ankunftszeit, an_prognose, an_prognose_status, abfahrtszeit, ab_prognose, ab_prognose_status, durchfahrt_tf, year, month):17:16", 'org.apache.hive.service.cli.operation.Operation:toSQLException:Operation.java:356', 'org.apache.hive.service.cli.operation.SQLOperation:prepare:SQLOperation.java:207', 'org.apache.hive.service.cli.operation.SQLOperation:runInternal:SQLOperation.java:261', 'o

In [16]:
# get stops 15km around ZH central station 
# since "We only consider departure and arrival stops in a 15km radius of Zürich's train station, Zürich HB (8503000), (lat, lon) = (47.378177, 8.540192)

In [33]:
# Step 1: create table of delays

query = f"""
    DROP TABLE IF EXISTS {username}.sbb_delays_ic_gen
"""
cur.execute(query)

query = f"""
    CREATE TABLE {username}.sbb_delays_ic_gen
    STORED AS ORC
    AS WITH Times AS(
        SELECT linien_id AS train, 
               unix_timestamp(ANKUNFTSZEIT, 'dd.MM.yyyy HH:mm') AS expected, 
               unix_timestamp(AN_PROGNOSE, 'dd.MM.yyyy HH:mm:ss') AS actual
        FROM {username}.sbb_orc
        WHERE verkehrsmittel_text = 'IC'
          AND year=2022
          AND linien_id IN ('702', '704', '706', '708', '710', '712', '714', '716', '718', '720', '722', '724', '726', '728', '730')
          AND haltestellen_name = 'Genève'
          AND upper(an_prognose_status) IN ('REAL', 'GESCHAETZT')
    )
    SELECT train, IF(actual > expected, actual - expected, 0) AS delay
    FROM Times
"""
cur.execute(query)

In [34]:
# Step 2: get the stats out for the median and 3rd (.75) quartile

query = f"""
    SELECT train,
           percentile_approx(delay, 0.5) AS second, 
           percentile_approx(delay, 0.75) AS third
    FROM {username}.sbb_delays_ic_gen
    GROUP BY train
    ORDER BY train
"""
df_delays_ic_gen = pd.read_sql(query, conn)

In [35]:
df_delays_ic_gen.head()

,train,second,third
0,702,0.0,34.5
1,704,5.0,75.0
2,706,0.0,64.0
3,708,0.0,43.5
4,710,0.0,49.5
